# latex.convert

> Convert LaTeX files into Obsidian.md notes

This module contains functions and methods to automatically make Obsidian notes from LaTeX files of mathematical papers, most notably those on arXiv.

See the [Potential Problems](#potential-problems) section below for some common errors that arise from this module and how to circumvent them.

In [ ]:
#| default_exp latex.convert

In [ ]:
#| export
from collections import OrderedDict
from itertools import product
import os
from os import PathLike
from pathlib import Path
import re
from typing import Union

from pylatexenc import latexwalker, latex2text
from pylatexenc.latexwalker import (
    LatexWalker, LatexEnvironmentNode, get_default_latex_context_db,
    LatexNode, LatexSpecialsNode, LatexMathNode, LatexMacroNode, LatexCharsNode,
    LatexGroupNode, LatexCommentNode
)
from pylatexenc.latex2text import (
    MacroTextSpec, EnvironmentTextSpec)
from pylatexenc.macrospec import (
    MacroSpec, LatexContextDb, EnvironmentSpec
)
import regex

from trouver.helper import (
    find_regex_in_text, dict_with_keys_topologically_sorted,
    containing_string_priority, replace_string_by_indices, text_from_file
)
from trouver.markdown.markdown.file import (
    MarkdownFile, MarkdownLineEnum
)

from trouver.markdown.obsidian.vault import VaultNote
from trouver.markdown.obsidian.personal.index_notes import (
    correspond_headings_with_folder, convert_title_to_folder_name
)
from trouver.markdown.obsidian.personal.reference import setup_folder_for_new_reference
from trouver.markdown.obsidian.vault import VaultNote
import warnings

In [ ]:
#| export
DEFAULT_NUMBERED_ENVIRONMENTS = ['theorem', 'corollary', 'lemma', 'proposition',
                                 'definition', 'conjecture', 'remark', 'example',
                                 'question']

In [ ]:
from fastcore.test import ExceptionExpected, test_eq
from trouver.helper import _test_directory, non_utf8_chars_in_file

## Potential problems

The following are some frequently problems that arise when using this module:


#### UnicodeDecodeErrors arise when reading LaTeX files

By default, the `text_from_file` method in `trouver.helper` reads files and attempts to decode them in `utf-8`. If a LaTeX file has characters that cannot be decoded into `utf-8`, then a `UnicodeDecodeError` may be raised. In this case, one can find identify these characters using the `trouver.helper.non_utf8_chars_in_file` method and modify the LaTeX file manually. It may be useful to use a text editor to jump to the positions that the characters are at and to change the encoding of the LaTeX file into `utf-8`; for example, the author of `trouver` has opened some `ANSI`-encoded LaTeX documents in `Notepad++` and converted their encoding into `UTF-8`.

In [ ]:
latex_file_path = _test_directory() / 'latex_examples' / 'circumflex_E_example.tex'
contents, non_utf8_chars = non_utf8_chars_in_file(latex_file_path)
#print(contents.decode(encoding='utf-8'))
test_eq(len(non_utf8_chars), 4)
print(f'The following are the non unicode characters and their positions: {non_utf8_chars}')

The following are the non unicode characters and their positions: [('Ê', 130), ('Ê', 165), ('Ë', 196), ('Ì', 227)]


#### `NoDocumentNodeErrors` arise even though the LaTeX file has a document environemt (i.e. `\begin{document}...\end{document}`)

The `find_document_node` method in this module sometimes is not able to detect the docment environment of a LaTeX file. This error is known to arise when
- there are macros (which include commands) defined that represents/expands to characters including `\begin{...}... \end{...}`. For example

In [ ]:
# TODO in the above explanation, include an example.

## LaTeX comments

In [ ]:
#| export
def remove_comments(text: str) -> str:
    # Find all occurrences of the comment pattern %[^\n]*
    return re.sub(r"%[^\n]*", "", text)

In [ ]:
text = r"""% Commands with parameters
\newcommand{\field}[1]{\mathbb{#1}}
\newcommand{\mat}[4]{\left[\begin{array}{cc}#1 & #2 \\
                                         #3 & #4\end{array}\right]}
\newcommand{\dual}[1]{#1^{\vee}}
\newcommand{\compl}[1]{\hat{#1}}
"""
assert '%' not in remove_comments(text)
print(remove_comments(text))

text = r"""Hi. I'm not commented. %But I am!"""
test_eq(remove_comments(text), "Hi. I'm not commented. ")


\newcommand{\field}[1]{\mathbb{#1}}
\newcommand{\mat}[4]{\left[\begin{array}{cc}#1 & #2 \\
                                         #3 & #4\end{array}\right]}
\newcommand{\dual}[1]{#1^{\vee}}
\newcommand{\compl}[1]{\hat{#1}}



## Divide LaTeX file into parts

To make Obsidian notes from a LaTeX file, I use sections/subsections, and environments as places to make new notes.

Things to think about:
Sections/subsections
environments, including theorems, corollaries, propositions, lemmas, definitions, notations
citations
Macros defined in the preamble?

LatexMacroNodes include: sections/subsections, citations, references, and labels, e.g.

```latex
> \section{Introduction}
\cite{ellenberg2nilpotent}
\subsection{The section conjecture}
\'e
\ref{fundamental-exact-sequence}
\cite{stix2010period}
\ref{fundamental-exact-sequence}
\cite{stix2012rational}
\cite[Appendix C]{stix2010period}
\subsection{The tropical section conjecture}
\label{subsec:tropical-section-conjecture}
```

#### Divide the preamble from the rest of the document

Some macros and commands defined in the preamble seem to prevent the `pylatexenc` methods from properly identifying the document environment/node in a LaTeX document. To circumvent this, we define a function to divide the preamble from the rest of the document

In [ ]:
#| export
def divide_preamble(
        text: str, # LaTeX document
        document_environment_name: str = "document"
        ) -> tuple[str, str]:
    """Divide the preamble from the rest of a LaTeX document.
    """
    begin_environment_str = rf'\begin{{{document_environment_name}}}'
    pattern = re.compile(re.escape(begin_environment_str))
    match = re.search(pattern, text) 
    start_match, end_match = match.span()
    return text[:start_match], text[start_match:]

    

In [ ]:
latex_file_path = _test_directory() / 'latex_examples' / 'example_with_a_command_with_begin.tex'
text = text_from_file(latex_file_path)

preamble, document = divide_preamble(text)
assert r'\begin{displaymath}' in preamble
assert r'Hyun Jong Kim' in preamble

assert r'Hyun Jong Kim' not in document
assert document.startswith(r'\begin{document}')
assert document.endswith('\\end{document}')

#### Get the Document Node

In [ ]:
#| export
class NoDocumentNodeError(Exception):
    """Exception raised when a LatexEnvironmentNode corresponding to the document 
    environment is expected in a LaTeX string, but no such node exists.
    
    **Attributes**
    - text - str
        - The text in which the document environment is not found.
    """
    
    def __init__(self, text):
        self.text = text
        super().__init__(
            f"The following text does not contain a document environment:\n{text}")



In [ ]:
#| export
def find_document_node(
        text: str, # LaTeX str
        document_environment_name: str = "document" # The name of the document environment.
        ) -> LatexEnvironmentNode:
    """Find the `LatexNode` object for the main document in `text`.
    
    **Raises**
    - NoDocumentNodeError
        - If document environment node is not detected.
    """
    w = LatexWalker(text)
    nodelist, _, _ = w.get_latex_nodes(pos=0)
    for node in nodelist:
        if node.isNodeType(LatexEnvironmentNode)\
                and node.environmentname == document_environment_name:
            return node
    raise NoDocumentNodeError(text)

The main content of virtually all LaTeX math articles belongs to a document environment, which pylatexenc can often detect. The `find_document_node` function returns this `LatexEnvironmentNode` object:

In [ ]:
latex_file_path = _test_directory() / 'latex_examples' / 'latex_example_1' / 'main.tex'
text = text_from_file(latex_file_path)
document_node = find_document_node(text)

If the LaTeX file has no `document` environment, then a `NoDocumentNodeError` is raised:

In [ ]:
# This latex document has its `document` environment commented out.
latex_file_path = _test_directory() / 'latex_examples' / 'latex_example_2' / 'main.tex'
text = text_from_file(latex_file_path)
with ExceptionExpected(NoDocumentNodeError):
    document_node = find_document_node(text)

At the time of this writinga `NoDocumentNodeError` may be raised even if the LaTeX file has a proper `document` environment

In [ ]:
latex_file_path = _test_directory() / 'latex_examples' / 'example_with_a_command_with_begin.tex'
text = text_from_file(latex_file_path)

# Perhaps in the future, pylatexenc will be able to find the document node for this file.
# When that time comes, delete this example.
with ExceptionExpected(NoDocumentNodeError):
    find_document_node(text)



The `divide_preamble` function can be used to circumvent this problem:

In [ ]:
preamble, document = divide_preamble(text)
document_node = find_document_node(document)
test_eq(document_node.environmentname, 'document')
assert document_node.isNodeType(LatexEnvironmentNode)

In [ ]:
# hide
# Find no document node error causes

# latex_file_path = r'_tests\latex_full\litt_cfag\main.tex'
# text = text_from_file(latex_file_path)
# document_node = find_document_node(text)

### Detect environment names used in a file

In [ ]:
#| export
def environment_names_used(
        text: str # LaTeX document
        ) -> set[str]: # The set of all environment names used in the main document.
    """Return the set of all environment names used in the main document
    of the latex code.
    """
    document_node = find_document_node(text)
    return {node.environmentname for node in document_node.nodelist
            if node.isNodeType(LatexEnvironmentNode)}        

Writers often use different environment names. For examples, writers often use `theorem`, `thm`, or `theo` for theorem environments or `lemma` or `lem` for lemma environments. The `environment_names_used` function returns the environment names actually used in the tex file.

In the example below, note that only the environments that are actually used are returned. For instance, the preamble of the document defines the theorem environments `problem`, and `lemma` (among other things), but these are not actually used in the document itself.

In [ ]:
latex_file_path = _test_directory() / 'latex_examples' / 'has_fully_written_out_environment_names.tex'
sample_text_1 = text_from_file(latex_file_path)
sample_output_1 = environment_names_used(sample_text_1)
test_eq({'corollary', 'proof', 'maincorollary', 'abstract', 'proposition'}, sample_output_1)

The document in the example below uses shorter names for theorem environments:

In [ ]:
latex_file_path = _test_directory() / 'latex_examples' / 'has_shorter_environment_names.tex'
sample_text_2 = text_from_file(latex_file_path)
sample_output_2 = environment_names_used(sample_text_2)
test_eq({'conj', 'notation', 'corollary', 'defn'}, sample_output_2)

#### Identify the numbering convention of a LaTeX document

LaTeX documents have various number conventions. Here are some examples of papers on the arXiv and notes on their numbering schemes. Note that the source code to these articles are publicly available on the arXiv. 

- Ellenberg, Venkatesh, and Westerland, *[Homological stability for Hurwitz spaces and the Cohen-Lenstra conjecture over function fields](https://arxiv.org/abs/0912.0325)*, 
    - The subsections and theorem-like environments of each section share a numbering scheme, e.g. section 1 has subsection `1.1 The Cohen-Lenstra heuristics`, `1.2 Theorem`, `1.3 Hurwitz spaces`. This is accomplished by defining theorem-like environments using the `subsection` counter, e.g.

        ```latex
        \theoremstyle{plain}
        \newtheorem{thm}[subsection]{Theorem}
        \newtheorem{prop}[subsection]{Proposition}
        \newtheorem{cor}[subsection]{Corollary}
        \newtheorem{remark}{Remark}
        \newtheorem{conj}[subsection]{Conjecture}
        \newtheorem*{conj*}{Conjecture}
         ```

        defines the `thm`, `prop`, `cor`, and `conj` environments to be numbered using the `subsection` counter, the `remark` environmment to be defiend as an unnumbered environment, and the `conj*` environment to be defined as an unnumbered environment with a different name than the `conj` environment.

    - The `\swapnumbers` command is included in the preamble to change the way that theorems are numbered in the document, e.g. the article has `1.2 Theorem` as opposed to `Theorem 1.2`.
    - The equations are numbered along the subsections - this is accomplished by the lines 

        ```latex
        \numberwithin{equation}{subsection}
        \renewcommand{\theequation}{\thesubsection.\arabic{equation}}
        ```

        in the preamble.
- Hoyois, *[A quadratic refinement of the Grothendieck-Lefschetz-Verdier Trace Formula](https://arxiv.org/abs/1309.6147)*
    - The theorem-like environments are numbered `Theorem 1.1, Theorem 1.3, Corollary 1.4, Theorem 1.5`, etc.
        - The theorem-like environments that are numbered are assigned the `equation` counter. In particular, the equation
        environments share their numberings with the theorem-like environments. For example, section 1 has Equation `(1.2)`
        - This equation counter is reset at the beginning of each section and the section number is included in the numbering via
        ```latex 
        \numberwithin{equation}{section}
        ```

In [ ]:
# TODO: consider different arxiv articles to see how they are numbered
# TODO: cosider swapnumbers
# TODO: consider numberwithin

In [ ]:
#| export
def numbered_newtheorems_counters_in_preamble(
        document: str # The LaTeX document
        ) -> dict[str, str]: # The keys are the command names of the environments. The values are the counters that the environments belong to, which can be custom defined or predefined in LaTeX.
    r"""Return the dict specifying the numbered `\newtheorem` command invocations

    Assumes that

    - invocations of the `\newtheorem` command are exclusively in the
    preamble of the LaTeX document.
    - theorem-like environments are defined using the `\newtheorem` command.
    - no environments of the same name are defined twice.

    This function does not take into account `numberwithins` being used.

    This function uses two separate regex patterns, one to detect the invocations of `\newtheorem`
    in which the optional parameter is the second parameter and one to detect those in which
    the optional parameter is the third parameter.


    """
    preamble, _ = divide_preamble(document)
    preamble = remove_comments(preamble)
    # TODO: maybe use the `regex` package instead of `re` with a recursive
    # balanced-curly braces detecting regex.
    second_parameter_pattern = re.compile(
        # In this case, the optional parameter (if any) should not follow the newtheorem.
        r'\\newtheorem\s*\{\s*(\w+)\s*\}\s*(\[\s*(\w+)\s*\])?\s*\{\s*(.*)\s*\}(?!\s*\[\s*(\w+)\s*\])')
    third_parameter_pattern = re.compile(
        r'\\newtheorem\s*\{\s*(\w+)\s*\}\s*\{\s*(.*)\s*\}\s*(\[\s*(\w+)\s*\])?')
    second_results = _search_counters_by_pattern(preamble, second_parameter_pattern, 3)
    third_results = _search_counters_by_pattern(preamble, third_parameter_pattern, 4)
    return second_results | third_results
    

def _search_counters_by_pattern(
        preamble: str,
        newtheorem_regex: re.Pattern,
        counter_group: int # This depends on which `newtheorem_regex` is used, and is either 3 or 4. 
        ) -> dict[str, str]:
    """
    Capture the newly defined theorem-like environment names as well as the
    counters that they belong to"""
    counters = {}
    for match in newtheorem_regex.finditer(preamble):
        env_name = match.group(1)
        counter = match.group(counter_group)
        # If no counter was specified, use the environment name as the counter
        if counter is None:
            counter = env_name
        counters[env_name] = counter
    return counters

In [ ]:
text = text_from_file(_test_directory() / 'latex_examples' / 'newtheorem_example.tex') 

counters = numbered_newtheorems_counters_in_preamble(text)
test_eq(counters, {'theorem': 'theorem', 'lemma': 'theorem', 'definition': 'theorem', 'corollary': 'corollary', 'remark': 'theorem'})


In [ ]:
text = r"""
\theoremstyle{plain}
\newtheorem{thm}[subsection]{Theorem}
\newtheorem{prop}[subsection]{Proposition}
\newtheorem{cor}[subsection]{Corollary}
\newtheorem{remark}{Remark}
\newtheorem{conj}[subsection]{Conjecture}
\newtheorem*{conj*}{Conjecture}
\begin{document}
\end{document}
"""
counters = numbered_newtheorems_counters_in_preamble(text)
test_eq(counters, {'thm': 'subsection', 'prop': 'subsection', 'cor': 'subsection', 'remark': 'remark', 'conj': 'subsection'})

`numbered_newtheorems_counters_in_preamble` ignores commented out text

In [ ]:
text = r"""
\theoremstyle{plain}
\newtheorem{thm}[subsection]{Theorem}
\newtheorem{prop}[subsection]{Proposition}
\newtheorem{cor}[subsection]{Corollary}
% \newtheorem{remark}{Remark}
\newtheorem{conj}[subsection]{Conjecture}
\newtheorem*{conj*}{Conjecture} %\newtheorem{fakeenv}{This won't be picked up!}
\begin{document}
\end{document}
"""
counters = numbered_newtheorems_counters_in_preamble(text)
test_eq(counters, {'thm': 'subsection', 'prop': 'subsection', 'cor': 'subsection', 'conj': 'subsection'})

`numbered_newtheorems_counters_in_preamble` does not account for `\numberwithin` command invocations.

In [ ]:
text = text_from_file(_test_directory() / 'latex_examples' / 'numbering_example_3_theorem_like_environments_share_counter_with_equation_and_reset_at_each_section' / 'main.tex')
print(text)
# So `numbered_newtheorems_counters_in_preamble` only considers the theorem-lik
#  environemnts as being counted by
# 'equation', even though the `\numberwithin{equation}{section}` ultimately counts them by
# `section`.
test_eq(numbered_newtheorems_counters_in_preamble(text), 
        {'theorem': 'equation',
         'proposition': 'equation',
         'lemma': 'equation',
         'corollary': 'equation',
         'definition': 'equation',
         'example': 'equation',
         'remark': 'equation'})

\documentclass{amsart}
\usepackage[utf8]{inputenc}
\usepackage{amsmath, amsfonts, amssymb, amsthm, amsopn}

\numberwithin{equation}{section}

\theoremstyle{plain}
\newtheorem*{theorem*}{Theorem}
\newtheorem*{theoremA}{Theorem A}
\newtheorem*{theoremB}{Theorem B}
\newtheorem{theorem}[equation]{Theorem}
\newtheorem{proposition}[equation]{Proposition}
\newtheorem{lemma}[equation]{Lemma}
\newtheorem{corollary}[equation]{Corollary}

\theoremstyle{definition}
\newtheorem{definition}[equation]{Definition}
\newtheorem{example}[equation]{Example}
\newtheorem*{acknowledgements}{Acknowledgements}
\newtheorem*{conventions}{Conventions}

\theoremstyle{remark}
\newtheorem{remark}[equation]{Remark}

\begin{document}

\section{Introduction}

\begin{theorem}
This is Theorem 1.1. This is because the \verb|\numberwithin{equation}{section}| makes the section number included in the equation counter and because the \\
\verb|\newtheorem{theorem}[equation]{Theorem}| command makes the environment \verb|theorem

In [ ]:
#| hide

# Test that the contents of the `counters_for_environments` function are detecting
# The defined commands correctly.
text = text_from_file(_test_directory() / 'latex_examples' / 'newtheorem_example.tex') 
preamble, _ = divide_preamble(text)
second_parameter_pattern = re.compile(
    # r'\\newtheorem\s*\{\s*(\w+)\s*\}\s*(\[\s*(\w+)\s*\])?\s*\{\s*(.*)\s*\}')
    r'\\newtheorem\s*\{\s*(\w+)\s*\}\s*(\[\s*(\w+)\s*\])?\s*\{\s*(.*)\s*\}(?!\s*\[\s*(\w+)\s*\])')
third_parameter_pattern = re.compile(
    r'\\newtheorem\s*\{\s*(\w+)\s*\}\s*\{\s*(.*)\s*\}\s*(\[\s*(\w+)\s*\])?')
second_results = _search_counters_by_pattern(preamble, second_parameter_pattern, 3)
third_results = _search_counters_by_pattern(preamble, third_parameter_pattern, 4)
assert 'remark' not in second_results
assert 'remark' in third_results

In [ ]:
#| export
def numberwithins_in_preamble(
        document: str # The LaTeX document
    ) -> dict[str, str]: # The keys are the first arguments of `numberwithin` invocations and the values ar ethe second arguments of `numberwithin` invocations.
    r"""Return the dict describing `numberwithin` commands invoked
    in the preamble of `document`."""
    preamble, _ = divide_preamble(document)
    preamble = remove_comments(preamble)
    pattern = regex.compile(r'\\numberwithin\s*\{\s*(\w+)\s*\}\s*\{\s*(.*)\s*\}')
    numberwithins = {}

    for match in pattern.finditer(preamble):
        environment_to_number = match.group(1)
        environment_to_count = match.group(2)
        numberwithins[environment_to_number] = environment_to_count

    

    return numberwithins

In [ ]:
text = text_from_file(_test_directory() / 'latex_examples' / 'numbering_example_3_theorem_like_environments_share_counter_with_equation_and_reset_at_each_section' / 'main.tex')
print(text)
test_eq(numberwithins_in_preamble(text), {'equation': 'section'})

\documentclass{amsart}
\usepackage[utf8]{inputenc}
\usepackage{amsmath, amsfonts, amssymb, amsthm, amsopn}

\numberwithin{equation}{section}

\theoremstyle{plain}
\newtheorem*{theorem*}{Theorem}
\newtheorem*{theoremA}{Theorem A}
\newtheorem*{theoremB}{Theorem B}
\newtheorem{theorem}[equation]{Theorem}
\newtheorem{proposition}[equation]{Proposition}
\newtheorem{lemma}[equation]{Lemma}
\newtheorem{corollary}[equation]{Corollary}

\theoremstyle{definition}
\newtheorem{definition}[equation]{Definition}
\newtheorem{example}[equation]{Example}
\newtheorem*{acknowledgements}{Acknowledgements}
\newtheorem*{conventions}{Conventions}

\theoremstyle{remark}
\newtheorem{remark}[equation]{Remark}

\begin{document}

\section{Introduction}

\begin{theorem}
This is Theorem 1.1. This is because the \verb|\numberwithin{equation}{section}| makes the section number included in the equation counter and because the \\
\verb|\newtheorem{theorem}[equation]{Theorem}| command makes the environment \verb|theorem

In [ ]:
# #| export 
# def counters_for_environments(
#         document: str # The LaTeX document    
#     ) -> dict[str, str]: # The keys are the environment names and the values are the counters for those environment names
#     r"""Return a dict mapping theorem-like environment names to their ultimate
#     counters.

#     This function takes into account `\numberwithin` command invocations

#     Assumes that the `\newtheorem` and `\numberedwithin` invocations are all in the
#     preamble. Also assumes that the counters are only specified either by
#     `\newtheorem` or `\numberedwithin` invocations.
#     """
#     newtheorem_counters = numbered_newtheorems_counters_in_preamble(document)
#     numberwithins = numberwithins_in_preamble(document)
#     ultimate_counters = newtheorem_counters.copy()
#     for environment_name, newtheorem_counter in newtheorem_counters.items():
#         if newtheorem_counter in numberwithins:
#             ultimate_counters[environment_name] = numberwithins[newtheorem_counter]
#     return ultimate_counters

In [ ]:
# text = text_from_file(_test_directory() / 'latex_examples' / 'numbering_example_3_theorem_like_environments_share_counter_with_equation_and_reset_at_each_section' / 'main.tex')
# print(text)
# test_eq(counters_for_environments(text), 
#         {'theorem': 'section',
#          'proposition': 'section',
#          'lemma': 'section',
#          'corollary': 'section',
#          'definition': 'section',
#          'example': 'section',
#          'remark': 'section'})

#### Getting the display names of environment

For example, `\newtheorem{theorem}{Theorem}` defines a theorem-like environment called `theorem` whose display name is `Theorem`.

In [ ]:
#| export
def display_names_of_environments(
        document: str # The LaTeX document
        ) -> dict[str, str]:  
    r"""Return the dict specifying the display names for each theorem-like environment.

    This function uses two separate regex patterns, one to detect the invocations of `\newtheorem`
    in which the optional parameter is the second parameter and one to detect those in which
    the optional parameter is the third parameter.

    Assumes that
    - invocations of the `\newtheorem` command are exclusively in the
    preamble of the LaTeX document.
    - theorem-like environments are defined using the `\newtheorem` command.
    - no environments of the same name are defined twice.

    """
    preamble, _ = divide_preamble(document)
    second_parameter_pattern = re.compile(
        # In this case, the optional parameter (if any) should not follow the newtheorem.
        r'\\newtheorem\*?\s*\{\s*(\w+\*?)\s*\}\s*(\[\s*(\w+)\s*\])?\s*\{\s*(.*)\s*\}(?!\s*\[\s*(\w+)\s*\])')
    third_parameter_pattern = re.compile(
        r'\\newtheorem\*?\s*\{\s*(\w+\*?)\s*\}\s*\{\s*(.*)\s*\}\s*(\[\s*(\w+)\s*\])?')
    second_results = _search_display_names_by_pattern(preamble, second_parameter_pattern, 4)
    third_results = _search_display_names_by_pattern(preamble, third_parameter_pattern, 2)
    return second_results | third_results
    

def _search_display_names_by_pattern(
        preamble: str,
        newtheorem_regex: re.Pattern,
        display_name_group: int # This depends on which `newtheorem_regex` is used, and is either 3 or 4. 
        ) -> dict[str, str]:
    """
    Capture the newly defined theorem-like environment names as well as the
    counters that they belong to"""
    display_names = {}
    for match in newtheorem_regex.finditer(preamble):
        env_name = match.group(1)
        display_name = match.group(display_name_group)
        display_names[env_name] = display_name
    return display_names

In [ ]:
text = text_from_file(_test_directory() / 'latex_examples' / 'newtheorem_example.tex') 
display_names = display_names_of_environments(text)
test_eq(display_names, {'theorem': 'Theorem',
 'lemma': 'Lemma',
 'definition': 'Definition',
 'corollary': 'Corollary',
 'conjecture*': 'Conjecture',
 'remark': 'Remark'})

### Divide latex text into parts

In [ ]:
#| export
def _setup_counters(
        numbertheorem_counters: dict[str, str]
        ) -> dict[str, int]:
    r"""
    Return a dict whose keys are of counters in the LaTeX document and whose
    values are all `0`. These key-value pairs are used to keep track of
    the numberings of `parts`.

    One special key is the key of the empty string `''`, which counters the
    parts which do not get a numbering, i.e. for most text that lie outside
    of (numbered) environments

    """
    # TODO: replace enumerated environments with markdown enumerated lists
    # and itemizes with markdown bulleted lists

    # cf. https://www.overleaf.com/learn/latex/Counters#Default_counters_in_LaTeX
    predefined_counters = [
        'part', # Incremented each time the `\part` command is used. It is not reset automatically and casn only be reset by the user
        'chapter', # Incremeneted each time the `\chapter` command is used.
        'section', # Incremented whenever a new `\section` command is encountered
        'subsection', # Incremented whenever a new `\subsection` command is encountered, reset whenever a new `\section` command is encountered
        'subsubsection', # Incremented whenever a new `\subsubsection` command is encounted, reset whenever a new `\subsection` or `\section` command is encountered
        'paragraph', # Incremeneted whenever a new paragraph is started. Reset whenever a new `\subsubsection`, `\subsection`, or `\section` command is encounted
        'subparagraph', # Incremented each time the `\subparagraph` command is used and reset at the beginning of a new
        'page', # Incremented each time a new page is started in the document
        'equation', # Incremeneted whenever the `\begin{equation}` environment is used. 
        'figure', # Incremented whenever a new `figure` environment is encountered
        'table', # Incremeneted whenever a new `taable` environment is encountered`
        'footnote', 
        'mpfootnote',
        'enumi',
        'enumii',
        'enumiii',
        'enumiv']

    counters = {counter: 0 for _, counter in numbertheorem_counters.items()}
    for counter in predefined_counters:
        counters[counter] = 0

    counters[''] = 0
    return counters

In [ ]:
sample_counters = _setup_counters(
    {'thm': 'subsection', 'prop': 'subsection', 'cor': 'subsection', 'remark': 'remark', 'conj': 'subsection'})
assert 'remark' in sample_counters
test_eq(sample_counters['remark'], 0)
assert 'thm' not in sample_counters  # 'thm' is an environment name, but not a counter.


In [ ]:
#| export
def _setup_numberwithins(
        explicit_numberwithins: dict[str, str]
        ) -> dict[str, str]: # The keys are counters and the values are all counters that the key is immediately numbered within.
    builtin_numberwithins = {
        'subsection': 'section',
        'subsubsection': 'subsection',
        'paragraph': 'subsubsection',
        'subparagraph': 'paragraph',
        'enumii': 'enumi',
        'enumiii': 'enumii',
        'enumiv': 'enumiii',
        'part': 'chapter',
        'appendix': 'chapter'
    }
    numberwithins = explicit_numberwithins | builtin_numberwithins
    return numberwithins

    

def _setup_all_numberwithins(
        explicit_numberwithins: dict[str, str]
        ) -> dict[str, list[str]]: # The keys are counters and the values are all counters that the key is numbered within.
    numberwithins = _setup_numberwithins(explicit_numberwithins) 
    all_counters = set()
    for key, value in numberwithins.items():
        all_counters.add(key)
        all_counters.add(value)
    all_numbered_withins = {counter: [] for counter in all_counters}
    for counter_1, counter_2 in product(all_counters, all_counters):
        if _is_numberedwithin(counter_1, counter_2, numberwithins):
            all_numbered_withins[counter_1].append(counter_2)
    return all_numbered_withins


def _is_numberedwithin(
        counter_1, counter_2, numberwithins: dict[str, str]
        ) -> bool:
    """Return `True` if `counter_1` is numbered within `counter_2""" 
    if counter_1 not in numberwithins:
        return False
    elif numberwithins[counter_1] == counter_2:
        return True
    return _is_numberedwithin(
        numberwithins[counter_1], counter_2, numberwithins)


In [ ]:
#| hide
sample_output = _setup_all_numberwithins({'equation': 'section'})
test_eq(sample_output['section'], [])
test_eq(sample_output['subsection'], ['section'])
test_eq(sample_output['equation'], ['section'])

sample_output = _setup_all_numberwithins({'theorem': 'section'})
test_eq(sample_output['theorem'], ['section'])

In [ ]:
#| hide
def _unnumbered_environments(
        numbertheorem_counters: dict[str, str],
        display_names: dict[str, str]) -> set[str]:
    r"""Return the set of unnumbered theorem-like environments defined by
    `\newtheorem`.
    """
    return {environment for environment in display_names
            if environment not in numbertheorem_counters}

    

In [ ]:
#| hide
sample_unnumbered_environments = _unnumbered_environments(
    {'theorem': 'theorem', 'lemma': 'theorem', 'definition': 'theorem', 'corollary': 'corollary', 'remark': 'theorem'},
    {'theorem': 'Theorem', 'lemma': 'Lemma', 'definition': 'Definition', 'corollary': 'Corollary', 'conjecture*': 'Conjecture', 'remark': 'Remark'} 
    )
test_eq(sample_unnumbered_environments, {'conjecture*'})

In [ ]:
#| hide
def _section_title(
        text: str
        ) -> tuple[bool, str]: # The bool is `True` if the section/subsection is numbered (i.e. is `section` or `subsection` as opposed to `section*` or `subsection*`). The `str` is the title of the section or subsection
    """Return the title of a section or subsection from a latex str
    and whether or not the section/subsection is numbered"""

    # Note that the `section` command has the optional argument `toc-title` which appears
    # in the table of contents, cf.
    # http://latexref.xyz/_005csection.html
    pattern = regex.compile(
        r'\\(?:section|subsection)\s*(?:\[.*\])?(\*)?\s*'
        r'\{((?>[^{}]+|\{(?2)\})*)\}',
        regex.MULTILINE
    )
    regex_search = regex.search(pattern, text)
    return regex_search.group(1) is None, regex_search.group(2)


In [ ]:
#| hide

# subsection, no extraneous spaces
sample_section = _section_title(r"\subsection{I am a subsection}")
test_eq(sample_section, (True, 'I am a subsection'))

# section, with extraneous spaces
sample_section = _section_title(r"\section {Generating series of special divisors}")
test_eq(sample_section, (True, 'Generating series of special divisors'))

# section, unnumbered
sample_section = _section_title(r"\section*{I am an unnumbered section}")
test_eq(sample_section, (False, 'I am an unnumbered section'))

# Subsection, unnumbered, extraneous spaces
sample_section = _section_title(r"\subsection*    {I am an unnumbered section and I have extraneous spaces}")
test_eq(sample_section, (False, 'I am an unnumbered section and I have extraneous spaces'))

# Multiline section
sample_section = _section_title(
    r"""\section*    {I am a section and I have span 
    multiple lines}""")
test_eq(sample_section, (False, 'I am a section and I have span \n    multiple lines'))

# Section with curly braces
sample_section = _section_title(
    r"""\section{ Can I talk about the finite field \mathcal{F}_p in this title?
        Can I also have multiple lines? Yes I can!}"""
)
test_eq(sample_section, (True, r""" Can I talk about the finite field \mathcal{F}_p in this title?
        Can I also have multiple lines? Yes I can!"""))

# Section with table of contents
sample_section = _section_title(
    r"\section [This is a Table of contents title] {This is the section title}"
)
test_eq(sample_section, (True, r"""This is the section title"""))


In [ ]:
#| export
def _is_section_node(node):
    return (node.isNodeType(LatexMacroNode)
            and node.macroname == 'section')

def _is_subsection_node(node):
    return (node.isNodeType(LatexMacroNode)
            and node.macroname == 'subsection')

def _is_environment_node(node):
    return node.isNodeType(LatexEnvironmentNode)

In [ ]:
#| hide
text = r"""
\documentclass{article}

\theoremstyle{plain}
\newtheorem{theorem}{Theorem}

\begin{document}

\section{This is section 1}

\subsection{This is subsection 1.1}

\begin{theorem}
\end{theorem}

\end{document}
"""
document_node = find_document_node(text)
assert _is_section_node(document_node.nodelist[1])
assert not _is_section_node(document_node.nodelist[3])
assert not _is_section_node(document_node.nodelist[5])

assert not _is_subsection_node(document_node.nodelist[1])
assert _is_subsection_node(document_node.nodelist[3])
assert not _is_subsection_node(document_node.nodelist[5])

assert not _is_environment_node(document_node.nodelist[1])
assert not _is_environment_node(document_node.nodelist[3])
assert _is_environment_node(document_node.nodelist[5])

# for node in document_node.nodelist:
#     print('\n')
#     print(node)
#     if node.isNodeType(LatexMacroNode):
#         print(node.macroname)
#     elif node.isNodeType(LatexEnvironmentNode):
#         print(node.environmentname)


In [ ]:
#| export
def _is_numbered(
        node: LatexNode,
        numbertheorem_counters: dict[str, str]
        ) -> bool:
    if _is_section_node(node) or _is_subsection_node(node):
        is_numbered, _ = _section_title(node.latex_verbatim())
        return is_numbered
    elif _is_environment_node(node):
        return node.environmentname in numbertheorem_counters
    else:
        return False

In [ ]:
#| hide
text = r"""
\documentclass{article}

\theoremstyle{plain}
\newtheorem{theorem}{Theorem}
\newtheorem*{theorem*}{Theorem}

\begin{document}
\begin{theorem}
\end{theorem}
\begin{theorem*}
\end{theorem*}
\end{document}
"""
document_node = find_document_node(text)
environments_to_counters = {'theorem': 'theorem'}

assert _is_numbered(document_node.nodelist[1], environments_to_counters)
assert not _is_numbered(document_node.nodelist[2], environments_to_counters)


# Example with numberwithin specified.
text = r"""
\documentclass{article}

\numberwithin{equation}{section}

\theoremstyle{plain}
\newtheorem{theorem}[equation]{Theorem}
\newtheorem*{theorem*}{Theorem}

\begin{document}
\begin{theorem}
\end{theorem}
\begin{theorem*}
\end{theorem*}
\end{document}
"""

document_node = find_document_node(text)
environments_to_counters = {'theorem': 'section'}

assert _is_numbered(document_node.nodelist[1], environments_to_counters)
assert not _is_numbered(document_node.nodelist[2], environments_to_counters)

# Example for sections and subsections
text = r"""
\begin{document}
\section{Section 1}
\subsection*{Unnumbered section}
\end{document}
"""
document_node = find_document_node(text)
environments_to_counters = {}

assert _is_numbered(document_node.nodelist[1], environments_to_counters)
assert not _is_numbered(document_node.nodelist[2], environments_to_counters)


In [ ]:
#| export
def _change_counters(
        node,
        counters,
        numbertheorem_counters: dict[str, str],
        all_numberwithins: dict[str, list[str]]
        ):
    # identify which counter to change
    # TODO
    # Take into consideration unnumbered non-environment node
    # Take into consideration unnumbered environment node
    if _is_environment_node(node):
        if node.environmentname in numbertheorem_counters:
           counter = numbertheorem_counters[node.environmentname] 
        else:
            counter = None
    elif _is_section_node(node):
        counter = 'section'
    elif _is_subsection_node(node):
        counter = 'subsection'
    else:
        counter = None

    # Section counters seem to only reset subsection counters
    # When the section is numbered, etc., cf. `numbering_example_4...`
    # and `numbering_example_5...` in `nbs\_tests\latex_examples`
    is_numbered = _is_numbered(node, numbertheorem_counters)
    # e.g. `\numberwithin{equation}{section}`` means that `equation` is
    # numbered within `section`, i.e. `equation` is reset whenever
    # `section` is incremeneted

    # if counter is None and not _is_environment_node(node):
    #     counters[''] += 1 
    #     return

    if is_numbered:
        counters[counter] += 1
    for numbered_counter, within_counter in all_numberwithins.items():
        if counter is not None and counter in within_counter:
            counters[numbered_counter] = 0



In [ ]:
#| export
def get_node_from_simple_text(text):
    w = LatexWalker(text)
    nodelist, _, _ = w.get_latex_nodes(pos=0)
    return nodelist[0]

In [ ]:
#| hide

text = r"""\begin{thm}This is a theorem. \end{thm}"""
node = get_node_from_simple_text(text)
# Test a theoreem being counted by its own counter.
numbertheorem_counters = {'thm': 'thm'}
all_numberwithins = {}
counters = {'thm': 1}
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'thm': 2})
# Test a theorem being countered by the equation counter.
numbertheorem_counters = {'thm': 'equation'}
all_numberwithins = {}
counters = {'equation': 2}
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'equation': 3})

text = r"""\begin{corollary}This is a corollary. \end{orollary}"""
node = get_node_from_simple_text(text)
# Test a theorem-like environment being counted by the counter of
# another theorem-like environment
numbertheorem_counters = {'corollary': 'theorem', 'theorem': 'theorem'}
all_numberwithins = {}
counters = {'theorem': 0}
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'theorem': 1})


# Test a theorem-like environment whose counter is numbered within
# The section counter.
# First, see what happens when a theorem is called
text = r"""\begin{theorem}This is a theorem. \end{theorem}"""
node = get_node_from_simple_text(text)
numbertheorem_counters = {'theorem': 'theorem'}
all_numberwithins = {'theorem': ['section']}
counters = {'section': 1, 'theorem': 0}
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'section': 1, 'theorem': 1})
# Next, see what happens when a new section is invoked:
text = r"""\section{New section! The theorem counter should be reset}"""
node = get_node_from_simple_text(text)
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'section': 2, 'theorem': 0})

# Test a theorem-like environment sharing a counter with equation
# and in turn equation is numbered within section.
text = r"""\begin{theorem}This is a theorem. \end{theorem}"""
node = get_node_from_simple_text(text)
numbertheorem_counters = {'theorem': 'equation'}
all_numberwithins = {'equation': ['section']}
counters = {'section': 1, 'equation': 0}
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'section': 1, 'equation': 1})
# Next, see what happens when a new section is invoked:
text = r"""\section{New section! The theorem counter should be reset}"""
node = get_node_from_simple_text(text)
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'section': 2, 'equation': 0})

# Test an unnumbered theorem-like environment counter
text = r"""\begin{thm*}This is a theorem. \end{thm*}"""
node = get_node_from_simple_text(text)
# Test a theoreem being counted by its own counter.
numbertheorem_counters = {'thm': 'thm'}
all_numberwithins = {}
counters = {'thm': 1}
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'thm': 1})

# Test a theorem-like environment sharing a counter with equation
# and in turn equation is numbered within section, but the 
# environment is unnumbered.
text = r"""\begin{theorem*}This is a theorem. \end{theorem*}"""
node = get_node_from_simple_text(text)
numbertheorem_counters = {'theorem': 'equation'}
all_numberwithins = {'equation': ['section']}
counters = {'section': 1, 'equation': 0}
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'section': 1, 'equation': 0})
# Next, see what happens when a unnumbered new section is invoked:
text = r"""\section*{New section! The theorem counter should be reset}"""
node = get_node_from_simple_text(text)
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'section': 1, 'equation': 0})

# Test the counter for text that does not belong to an environment
# In the current implementation of _change_counters, the '' counter
# is not actually changed.
text = r"""Just some text."""
node = get_node_from_simple_text(text)
numbertheorem_counters = {'theorem': 'equation'}
all_numberwithins = {'equation': ['section']}
counters = {'section': 1, 'equation': 0, '': 0}
_change_counters(node, counters, numbertheorem_counters, all_numberwithins)
test_eq(counters, {'section': 1, 'equation': 0, '': 0})

In [ ]:
#| export
def _node_numbering(
        node: LatexNode,
        numbertheorem_counters: dict[str, str],
        numberwithins: dict[str, str],
        counters: dict[str, int]
        ) -> str: # Just the numbering of the node, no "section/subsection" or displayname
    if _is_section_node(node):
        counter = 'section'
    elif _is_subsection_node(node):
        counter = 'subsection'
    elif _is_environment_node(node):
        counter = numbertheorem_counters[node.environmentname]
    return _numbering_helper('', counter, numberwithins, counters)


def _numbering_helper(
        trailing_numbering: str,
        counter: str,
        numberwithins: dict[str, str],
        counters: dict[str, int]
        ) -> str:
    """Recurisve helper function to `_node_numbering`."""
    if counter not in numberwithins and counter not in counters:
        return trailing_numbering
    if counter not in numberwithins and counter in counters and trailing_numbering:
        return f'{counters[counter]}.{trailing_numbering}'
    if counter not in numberwithins and counter in counters and not trailing_numbering:
        return f'{counters[counter]}'

    parent_counter = numberwithins[counter]
    current_count = counters[counter]
    if not trailing_numbering:
        to_pass_to_trailing_numbering = str(current_count)
    else:
        to_pass_to_trailing_numbering = f'{current_count}.{trailing_numbering}'

    return _numbering_helper(
        to_pass_to_trailing_numbering,
        parent_counter,
        numberwithins,
        counters)
    

In [ ]:
text = r"""\begin{thm}This is a theorem. \end{thm}"""
node = get_node_from_simple_text(text)
# Test a theoreem being counted by its own counter.
numbertheorem_counters = {'thm': 'thm'}
numberwithins = {}
counters = {'thm': 1}
sample_numbering = _node_numbering(
    node, numbertheorem_counters, numberwithins, counters)
test_eq(sample_numbering, '1')
# Test a theorem being countered by the equation counter.
numbertheorem_counters = {'thm': 'equation'}
numberwithins = {}
counters = {'equation': 2}
sample_numbering = _node_numbering(
    node, numbertheorem_counters, numberwithins, counters)
test_eq(sample_numbering, '2')
# Test a theorem being countered by the equation counter.
numbertheorem_counters = {'thm': 'equation'}
numberwithins = {}
counters = {'equation': 2}
sample_numbering = _node_numbering(
    node, numbertheorem_counters, numberwithins, counters)
test_eq(sample_numbering, '2')

text = r"""\begin{corollary}This is a corollary. \end{orollary}"""
node = get_node_from_simple_text(text)
# Test a theorem-like environment being counted by the counter of
# another theorem-like environment
numbertheorem_counters = {'corollary': 'theorem', 'theorem': 'theorem'}
numberwithins = {}
counters = {'theorem': 0}
sample_numbering = _node_numbering(
    node, numbertheorem_counters, numberwithins, counters)
test_eq(sample_numbering, '0')

# Test a theorem-like environment whose counter is numbered within
# The section counter.
# First, see what happens when a theorem is called
text = r"""\begin{theorem}This is a theorem. \end{theorem}"""
node = get_node_from_simple_text(text)
numbertheorem_counters = {'theorem': 'theorem'}
numberwithins = {'theorem': 'section'}
counters = {'section': 1, 'theorem': 0}
sample_numbering = _node_numbering(
    node, numbertheorem_counters, numberwithins, counters)
test_eq(sample_numbering, '1.0')

# Next, see what happens when a new section is invoked:
text = r"""\section{New section! The theorem counter should be reset}"""
node = get_node_from_simple_text(text)
sample_numbering = _node_numbering(
    node, numbertheorem_counters, numberwithins, counters)
test_eq(sample_numbering, '1')

# Test a theorem-like environment sharing a counter with equation
# and in turn equation is numbered within section.
text = r"""\begin{theorem}This is a theorem. \end{theorem}"""
node = get_node_from_simple_text(text)
numbertheorem_counters = {'theorem': 'equation'}
numberwithins = {'equation': 'section'}
counters = {'section': 1, 'equation': 0}
sample_numbering = _node_numbering(
    node, numbertheorem_counters, numberwithins, counters)
test_eq(sample_numbering, '1.0')
# Next, see what happens when a new section is invoked:
text = r"""\section{New section! The theorem counter should be reset}"""
node = get_node_from_simple_text(text)
sample_numbering = _node_numbering(
    node, numbertheorem_counters, numberwithins, counters)
test_eq(sample_numbering, '1')

In [ ]:
#| export

def _title(
        node: LatexNode,
        numbertheorem_counters: dict[str, str],
        numberwithins: dict[str, str], # An output of _setup_numberwithins
        all_numberwithins: dict[str, list[str]], # An output of all_numberwithins
        display_names: dict[str, str],
        counters: dict[str, int],
        swap_numbers: bool):
    numbered = _is_numbered(node, numbertheorem_counters)
    if _is_section_node(node) and numbered:
        _, title = _section_title(node.latex_verbatim())
        return f"{counters['section']}. {title}"
    if _is_section_node(node) and not numbered:
        _, title = _section_title(node.latex_verbatim())
        return title 

    if _is_subsection_node(node) and numbered:
        _, title = _section_title(node.latex_verbatim())
        return f"{counters['section']}.{counters['subsection']}. {title}"
    if _is_subsection_node(node) and not numbered:
        _, title = _section_title(node.latex_verbatim())
        return title

    if _is_environment_node(node):
        return _title_for_environment_node(
            node, numbertheorem_counters, numberwithins,
            display_names, counters, swap_numbers)


def _title_for_environment_node(
        node: LatexNode,
        numbertheorem_counters: dict[str, str],
        numberwithins: dict[str, list[str]],
        display_names: dict[str, str],
        counters: dict[str, int],
        swap_numbers: bool):
    numbered = _is_numbered(node, numbertheorem_counters)
    # TODO: see what happens when environments are numbered within
    # sections vs. subsections
    if not numbered:
        numbering = None
    else:
        numbering = _node_numbering(
            node, numbertheorem_counters, numberwithins, counters)
    
    environment = node.environmentname
    if not numbered:
        return display_names[environment]
    elif swap_numbers:
        return f'{numbering}. {display_names[environment]}.'
    else:
        return f'{display_names[environment]} {numbering}.'
        

In [ ]:
#| hide

# Theorem that is not numbered within anything
text = r"""\begin{thm}This is a theorem. \end{thm}"""
node = get_node_from_simple_text(text)
numbertheorem_counters = {'thm': 'thm'}
numberwithins = {}
all_numberwithins = {}
display_names = {'thm': 'Theorem'}
counters = {'thm': 1}
swap_numbers = False
sample_title = _title(
    node, numbertheorem_counters, numberwithins, all_numberwithins,
    display_names, counters, swap_numbers)
test_eq(sample_title, 'Theorem 1.')

swap_numbers = False
sample_title = _title(
    node, numbertheorem_counters, numberwithins, all_numberwithins,
    display_names, counters, swap_numbers)
test_eq(sample_title, 'Theorem 1.')

# Theorem that is counted by equation, which in turn is numbered within
# section
text = r"""\begin{thm}This is a theorem. \end{thm}"""
node = get_node_from_simple_text(text)
numbertheorem_counters = {'thm': 'equation'}
numberwithins = {'equation': 'section'}
all_numberwithins = {'equation': ['section']}
display_names = {'thm': 'Theorem'}
counters = {'equation': 1, 'section': 2}
swap_numbers = False
sample_title = _title(
    node, numbertheorem_counters, numberwithins, all_numberwithins,
    display_names, counters, swap_numbers)
test_eq(sample_title, 'Theorem 2.1.')

swap_numbers = True
sample_title = _title(
    node, numbertheorem_counters, numberwithins, all_numberwithins,
    display_names, counters, swap_numbers)
test_eq(sample_title, '2.1. Theorem.')

# Section
text = r"""\section{This is a section}"""
node = get_node_from_simple_text(text)
numbertheorem_counters = {'thm': 'equation'}
numberwithins = {'equation': 'section'}
all_numberwithins = {'equation': ['section']}
display_names = {'thm': 'Theorem'}
counters = {'equation': 1, 'section': 2}
swap_numbers = False
sample_title = _title(
    node, numbertheorem_counters, numberwithins, all_numberwithins,
    display_names, counters, swap_numbers)
test_eq(sample_title, '2. This is a section')

swap_numbers = True
sample_title = _title(
    node, numbertheorem_counters, numberwithins, all_numberwithins,
    display_names, counters, swap_numbers)
test_eq(sample_title, '2. This is a section')

# Subsection
text = r"""\subsection{This is a subsection}"""
node = get_node_from_simple_text(text)
numbertheorem_counters = {'thm': 'equation'}
numberwithins = {'equation': 'section', 'subsection': 'section'}
all_numberwithins = {'equation': ['section'], 'subsection': ['section']}
display_names = {'thm': 'Theorem'}
counters = {'equation': 1, 'section': 2, 'subsection': 3}
swap_numbers = False
sample_title = _title(
    node, numbertheorem_counters, numberwithins, all_numberwithins,
    display_names, counters, swap_numbers)
test_eq(sample_title, '2.3. This is a subsection')

swap_numbers = True
sample_title = _title(
    node, numbertheorem_counters, numberwithins, all_numberwithins,
    display_names, counters, swap_numbers)
test_eq(sample_title, '2.3. This is a subsection')

In [ ]:
#| export
def swap_numbers_invoked(
        preamble: str
        ) -> bool: # 
    """Returns `True` if `\swapnumbers` is in the preamble.

    Assume that a mention of `\swapnumbers` is an actual invocation.
    """
    preamble = remove_comments(preamble)
    return '\swapnumbers' in preamble

In [ ]:
assert swap_numbers_invoked('\swapnumbers')
assert not swap_numbers_invoked(r'''
\documentclass{article}
\usepackage{amsthm}
%\usepackage{amsmath}

\newtheorem{theorem}{Theorem} % \swapnumbers
\newtheorem{corollary}[theorem]{Corollary}
\newtheorem{definition}[theorem]{Definition}
\newtheorem*{remark*}{Remark}''')

In [ ]:
#| export
def divide_latex_text(
        document: str, 
        # environments_to_divide_along: list[str], # A list of the names of environments that warrant a new note
        # numbered_environments: list[str], # A list of the names of environments which are numbered in the latex code. 
        environments_to_not_divide_along: list[str] = ['equation', 'equation*', 'proof', 'align', 'align*'], # A list of the names of the environemts along which to not make a new note, unless the environment starts a section (or the entire document).
        ) -> list[tuple[str, str]]: # Each tuple is of the form (<note_type_and_or_numbering>, <text>)
    r"""Divide LaTeX text to convert into Obsidian.md notes.

    Assumes that the counters in the LaTeX document are either the
    predefined ones or specified by the `\newtheorem` command.

    This function does not divide out `\subsubsection`'s

    TODO: Implement counters specified by `\newcounter`, cf. 
    https://www.overleaf.com/learn/latex/Counters#LaTeX_commands_for_working_with_counters.
    """
    numbertheorem_counters = numbered_newtheorems_counters_in_preamble(document)
    explicit_numberwithins = numberwithins_in_preamble(document)
    numberwithins = _setup_numberwithins(explicit_numberwithins)
    all_numberwithins = _setup_all_numberwithins(explicit_numberwithins)
    # environments_to_counters = counters_for_environments(document)
    display_names = display_names_of_environments(document)
    counters = _setup_counters(numbertheorem_counters)
    unnumbered_environments = _unnumbered_environments(
        numbertheorem_counters, display_names)
    # Eventually gets returned
    preamble, main_document = divide_preamble(document)
    document_node = find_document_node(main_document)
    swap_numbers = swap_numbers_invoked(preamble)
    parts = []
    # "Accumulates" a "part" until text that should comprise a new part is encountered
    accumulation = '' 
    for node in document_node.nodelist:
        accumulation = _process_node(
            node, environments_to_not_divide_along, accumulation,
            numbertheorem_counters,
            numberwithins, all_numberwithins, counters,
            display_names, swap_numbers, parts)
    _append_non_environment_accumulation_to_parts_if_non_empty(
        accumulation, counters, parts)
    return parts


def _process_node(
        node, environments_to_not_divide_along, accumulation,
        numbertheorem_counters,
        numberwithins, all_numberwithins, counters,
        display_names, swap_numbers, parts):
    """
    Update `accumulation`, `counter`, and `parts` based on the contents of `node`.

    Also return 'accumulation` to update it.
    """
    _change_counters(node, counters, numbertheorem_counters, numberwithins)

    if _node_warrants_new_part(
            node, environments_to_not_divide_along, accumulation, parts):
        accumulation =  _append_non_environment_accumulation_to_parts_if_non_empty(
            accumulation, counters, parts)
        
        parts.append([
            _title(
                node, numbertheorem_counters, numberwithins, all_numberwithins,
                display_names, counters, swap_numbers),
            node.latex_verbatim()])
    else:
        accumulation += node.latex_verbatim()
        # In _change_counters`, the '' counter is incremented by default.
        # This offsets the incorrectly incrementation.
    return accumulation

    # if (_is_section_node(node)
    #         or _is_subsection_node(node)
    #         or _is_environment_node(node)):
    #     accumulation =  _append_non_environment_accumulation_to_parts_if_non_empty(
    #         accumulation, counters, parts)
        
    #     parts.append([
    #         _title(
    #             node, numbertheorem_counters, numberwithins, all_numberwithins,
    #             display_names, counters, swap_numbers),
    #         node.latex_verbatim()])
    # else:
    #     accumulation += node.latex_verbatim()
    #     # In _change_counters`, the '' counter is incremented by default.
    #     # This offsets the incorrectly incrementation.
    # return accumulation


def _append_non_environment_accumulation_to_parts_if_non_empty(
        accumulation: str, counters, parts):
    """Append accumulation to `parts` if `accumulation` is nonempty
    and return the updated `accumulation` """
    if accumulation.strip() != '':
        counters[''] += 1
        parts.append([str(counters['']), accumulation.strip()])
        return ''
    else:
        return accumulation


def _node_warrants_new_part(
    # TODO: write tests
        node, environments_to_not_divide_along: list[str],
        accumulation: str, parts: list) -> bool:
    if _is_section_node(node) or _is_subsection_node(node):
        return True
    elif not _is_environment_node(node):
        return False
    # Is environment node from here and below.
    if len(parts) == 0 and accumulation.strip() == '':
        return True
    previous_node = get_node_from_simple_text(parts[-1][1])
    if (accumulation.strip() == ''
            and (_is_section_node(previous_node)
                 or _is_subsection_node(previous_node))):
        return True
    return node.environmentname not in environments_to_not_divide_along


The `divide_latex_text` function divides latex text 

In [ ]:
file = _test_directory() / 'latex_examples' / 'numbering_example_1_consecutive_numbering_scheme' / 'main.tex'
text = text_from_file(file)
sample_output = divide_latex_text(text)
print(sample_output)
assert sample_output[0][0] == '1'
assert sample_output[1][0] == '1. Introduction'
assert sample_output[2][0] == 'Theorem 1.'
assert sample_output[3][0] == 'Corollary 2.'
assert sample_output[4][0] == 'Remark'


[['1', 'For this document, the `theorem` counter is not reset whenever a new section begins.\n\nA similar numbering scheme can be accomplished by importing \\verb|amsmath| and invoking the code \\verb|\\numberwithin{theorem}{part}| in the preamble.'], ['1. Introduction', '\\section{Introduction}'], ['Theorem 1.', '\\begin{theorem}\nThis is Theorem 1.\n\\end{theorem}'], ['Corollary 2.', '\\begin{corollary}\nThis is Corollary 2.\n\\end{corollary}'], ['Remark', '\\begin{remark*}\nThis is a remark. It is unnumbered and it does not affect the numberings of other environments.\n\\end{remark*}'], ['Definition 3.', '\\begin{definition}\nThis is Definition 3.\n\\end{definition}'], ['2. Another Section', '\\section{Another Section}'], ['Theorem 4.', '\\begin{theorem}\nThis is Theorem 4.\n\\end{theorem}'], ['2', 'And we might get a corollary!'], ['Corollary 5.', '\\begin{corollary}\nThis is Corollary 5.\n\\end{corollary}'], ['Definition 6.', '\\begin{definition}\nThis is Definition 6.\n\\end{defi

In [ ]:
file = _test_directory() / 'latex_examples' / 'numbering_example_2_numbering_scheme_reset_at_each_section' / 'main.tex'
text = text_from_file(file)
print(divide_latex_text(text))

[['1', 'This document resets its `theorem` counter whenever a new section begins.'], ['1. Introduction', '\\section{Introduction}'], ['Theorem 1.1.', '\\begin{theorem}\nThis is Theorem 1.1.\n\\end{theorem}'], ['Corollary 1.2.', '\\begin{corollary}\nThis is Corollary 1.2.\n\\end{corollary}'], ['Remark', '\\begin{remark*}\nThis is a remark. It is unnumbered and it does not affect the numberings of other environments.\n\\end{remark*}'], ['Definition 1.3.', '\\begin{definition}\nThis is Definition 1.3.\n\\end{definition}'], ['2. Another Section', '\\section{Another Section}'], ['Theorem 2.1.', '\\begin{theorem}\nThis is Theorem 2.1.\n\\end{theorem}'], ['Corollary 2.2.', '\\begin{corollary}\nThis is Corollary 2.2.\n\\end{corollary}'], ['Definition 2.3.', '\\begin{definition}\nThis is Definition 2.3.\n\\end{definition}']]


In [ ]:
file = _test_directory() / 'latex_examples' / 'numbering_example_3_theorem_like_environments_share_counter_with_equation_and_reset_at_each_section' / 'main.tex'
text = text_from_file(file)
print(divide_latex_text(text))

[['1. Introduction', '\\section{Introduction}'], ['Theorem 1.1.', '\\begin{theorem}\nThis is Theorem 1.1. This is because the \\verb|\\numberwithin{equation}{section}| makes the section number included in the equation counter and because the \\\\\n\\verb|\\newtheorem{theorem}[equation]{Theorem}| command makes the environment \\verb|theorem| be counted by the equation counter.\n\\end{theorem}'], ['1', 'The following makes an equation labeled 1.2; \n\\begin{equation}\n5 + 7 = 12\n\\end{equation}'], ['Theorem', '\\begin{theorem*}\nThis Theorem is unnumbered\n\\end{theorem*}'], ['Corollary 1.2.', '\\begin{corollary}\nThis is Corollary 1.3.\n\\end{corollary}'], ['2. Another section', '\\section{Another section}'], ['Theorem 2.1.', '\\begin{theorem}\nThis is theorem 2.1\n\\end{theorem}'], ['2', 'The following is labeled 2.2:\n\\begin{equation}\n3+5 = 8.\n\\end{equation}']]


In [ ]:
file = _test_directory() / 'latex_examples' / 'numbering_example_4_unnumbered_section' / 'main.tex'
text = text_from_file(file)
print(divide_latex_text(text))

[['1. This is section 1', '\\section{This is section 1}'], ['Theorem 1.1.', '\\begin{theorem}\nThis is Theorem 1.1.\n\\end{theorem}'], ['1.1. This is a subsection 1.1', '\\subsection{This is a subsection 1.1}'], ['1', 'The following makes an equation labeled 1; \n\\begin{equation}\n5 + 7 = 12\n\\end{equation}'], ['Theorem', '\\begin{theorem*}\nThis Theorem is unnumbered\n\\end{theorem*}'], ['1.2. This is subsection 1.2', '\\subsection{This is subsection 1.2}'], ['Corollary 1.2.', '\\begin{corollary}\nThis is Corollary 1.2.\n\\end{corollary}'], ['Unnumbered section', '\\section*{Unnumbered section}'], ['1.1. This is subsection 1.3', '\\subsection{This is subsection 1.3}'], ['2', '\\subsubsection{This is subsubsection 1.3.1}'], ['Theorem 1.1.', '\\begin{theorem}\nThis is Theorem 1.3.\n\\end{theorem}'], ['2. Another section', '\\section{Another section}'], ['2.1. This is subsection 2.1', '\\subsection{This is subsection 2.1}'], ['Theorem 2.1.', '\\begin{theorem}\nThis is Theorem 2.1\n\\en

In [ ]:
file = _test_directory() / 'latex_examples' / 'numbering_example_5_subsections_and_theorem_like_environments_share_counter' / 'main.tex'
text = text_from_file(file)
print(divide_latex_text(text))

[['1. This is section 1', '\\section{This is section 1}'], ['1.1. Theorem.', '\\begin{thm}\nThis is 1.1 Theorem. Note that the \\verb|\\swapnumbers| command is invoked in the preamble.\n\\end{thm}'], ['1.2. This is 1.2 subsection.', '\\subsection{This is 1.2 subsection.}'], ['1', 'Note that the equation counter is numbered within the subsection counter and that the theorem-like environments are numbered with the equation counter.\n\n\\subsubsection{This is 1.2.1 Subsubsection}'], ['1. Remark.', '\\begin{remark}\nThis is an unnumbered remark.\n\\end{remark}'], ['1.3. Proposition.', '\\begin{prop}\nThis is 1.3 Proposition.\n\\end{prop}'], ['Unnumbered section', '\\section*{Unnumbered section}'], ['1.1. Theorem.', '\\begin{thm}\nThis is 1.4 Theorem.\n\\end{thm}'], ['2. This is Section 2', '\\section{This is Section 2}'], ['2.1. Theorem.', '\\begin{thm}\nThis is 2.1 Theorem\n\\end{thm}']]


In [ ]:
# TODO: Find a list of environment names commonly used.

In [ ]:
# TODO: examples with different numbering convention and different numbered environments

In [ ]:
# TODO: make numbering_convention work correctly.
# Here are some latex files with different conventions:
# - All subsections in a section share numbering, 
#   - achter_pries_imht https://arxiv.org/abs/math/0608038: e.g. Lemmas 2.1, 2.2, 2.3 are in subsection 2.2 and Lemma 2.4 and Remark 2.5 are in subsection 2.4.as_integer_ratio
#   - pauli_wickelgren https://arxiv.org/abs/2010.09374: e.g. Example 3.5, 3.11 are in subsubsection 3.3.2, Exercise 4.1, Remark 4.2, are in subsection 4.1, Theorem 4.3 is in subsection 4.2, Theorem 4.4 is in subsection 4.3
# - Different environment types have different counts and the counts do not show the section number.
#   - vankataramana_imbrd https://arxiv.org/abs/1205.6543: 
#       - e.g. section 1 has Theorem 1, Remark 1, Remark 2, Remark 3, subsection 1.1.3 has Remark 4, Subsection 2.2 has Definition 1

## Formatting modifications

### Identify macros and commands to replace

Authors usually define a lot of custom commands and macros in their LaTeX files. Such customizations vary from author to author and most customized commands are not recognized by Obsidian. 

See `nbs/_tests/latex_examples/commands_example/main.tex` for some examples of custom commands.

In [ ]:
#| export
def custom_commands(
        preamble: str, # The preamble of a LaTeX document.
        ) -> list[tuple[str, int, Union[str, None], str]]: # Each tuple consists of 1. the name of the custom command 2. the number of parameters 3. The default argument if specified or `None` otherwise, and 4. the display text of the command.
    """
    Return a dict mapping commands (and math operators) defined in `preamble` to
    the number of arguments display text of the commands.

    Assumes that the newcommands only have at most one default parameter (newcommands with
    multiple default parameters are not valid in LaTeX).

    Ignores all comented newcommands.
    """
    preamble = remove_comments(preamble)
    newcommand_regex = regex.compile(
        r'(?<!%)\s*\\(?:(?:re)?newcommand|DeclareMathOperator)\s*\{\\\s*(\w+)\s*\}\s*(\[(\d+)\]\s*(?:\[(\w+)\])?)?\s*\{((?>[^{}]+|\{(?5)\})*)\}', re.MULTILINE)
    # newcommand_regex = regex.compile(
    #     r'(?<!%)\s*\\(?:re)?newcommand\s*\{\\\s*(\w+)\s*\}\s*(\[(\d+)\]\s*(?:\[(\w+)\])?)?\s*\{\s*(.*)\s*\}', re.MULTILINE)
    commands = []
    for match in newcommand_regex.finditer(preamble):
        name = match.group(1)
        num_args = match.group(3)
        optional_default_arg = match.group(4)
        definition = match.group(5)

        # Convert the number of arguments to an integer, if it was specified
        if num_args is not None:
            num_args = int(num_args)
        else:
            num_args = 0

        commands.append((name, num_args, optional_default_arg, definition))
    return commands



In [ ]:
# Basic
text_1 = r'\newcommand{\con}{\mathcal{C}}'
test_eq(custom_commands(text_1), [('con', 0, None, r'\mathcal{C}')])

# With a parameter
text_2 = r'\newcommand{\field}[1]{\mathbb{#1}}'
test_eq(custom_commands(text_2), [('field', 1, None, r'\mathbb{#1}')]) 

# With multiple parameters, the first of which has a default value of `2`
text_3 = r'\newcommand{\plusbinomial}[3][2]{(#2 + #3)^#1}'
test_eq(custom_commands(text_3), [('plusbinomial', 3, '2', r'(#2 + #3)^#1')])

# The display text has backslashes `\` and curly brances `{}``
text_4 = r'\newcommand{\beq}{\begin{displaymath}}'
test_eq(custom_commands(text_4), [('beq', 0, None, '\\begin{displaymath}')])


# Basic with spaces in the newcommand declaration
text_6 = r'\newcommand {\con}  {\mathcal{C}}'
test_eq(custom_commands(text_6), [('con', 0, None, r'\mathcal{C}')])

# With a parameter and spaces in the newcommand declaration
text_7 = r'\newcommand   {\field}   [1] {\mathbb{#1}}'
test_eq(custom_commands(text_7), [('field', 1, None, r'\mathbb{#1}')])

# With multiple parameters, a default value, and spaces in the newcommand declaration
text_8 = r'\newcommand {\plusbinomial} [3] [2] {(#2 + #3)^#1}'
test_eq(custom_commands(text_8), [('plusbinomial', 3, '2', r'(#2 + #3)^#1')]) 

# With a comment `%'; commented out command declarations should not be detected.
text_9 = r'% \newcommand{\con}{\mathcal{C}}'
test_eq(custom_commands(text_9), [])


# Spanning multiple lines
text_10 = r'''\newcommand{\mat}[4]{\left[\begin{array}{cc}#1 & #2 \\
                                         #3 & #4\end{array}\right]}'''
test_eq(
    custom_commands(text_10),
    [('mat', 4, None,
             '\\left[\\begin{array}{cc}#1 & #2 \\\\\n                                         #3 & #4\\end{array}\\right]')])

# Math operator
text_1 = r'\DeclareMathOperator{\Hom}{Hom}'
test_eq(custom_commands(text_1), [('Hom', 0, None, 'Hom')])

text_2 = r'\DeclareMathOperator{\tConf}{\widetilde{Conf}}'
test_eq(custom_commands(text_2), [('tConf', 0, None, r'\widetilde{Conf}')])

In [ ]:
# TODO: use a regexp pattern like this one to extract balanced curly braces
# \\mat\{((?>[^{}]+|\{(?1)\})*)\}\{((?>[^{}]+|\{(?2)\})*)\}

In [ ]:
#| export
def regex_pattern_detecting_command(
        command_tuple: tuple[str, int, Union[None, str], str], # Consists of 1. the name of the custom command 2. the number of parameters 3. The default argument if specified or `None` otherwise, and 4. the display text of the command.
        ) -> regex.Pattern:
    """Return a `regex.pattern` object (not a `re.pattern` object) detecting
    the command with the specified number of parameters, optional argument,
    and display text.

    Assumes that the curly braces used to write the invocations of the commands
    are balanced and properly nested. Assumes that there are no two commands
    of the same name.
    """
    command_name, num_parameters, optional_arg, _ = command_tuple
    backslash_name = fr"\\{command_name}"
    optional_argument_detection = fr"(?:\[(.*?)\])?" if optional_arg is not None else ""
    argument_detection = r""
    if optional_arg is not None:
        trailing_arguments = [_argument_detection(i) for i in range(2, 1+num_parameters)]
        trailing_args_pattern = "\\s*".join(trailing_arguments)
        pattern = (f"{backslash_name}\\s*{optional_argument_detection}\\s*{trailing_args_pattern}")
    elif num_parameters > 0:
        arguments = [_argument_detection(i) for i in range(1, 1+num_parameters)]
        args_pattern = "\\s*".join(arguments)
        pattern = f"{backslash_name}\\s*{args_pattern}"
    else:
        pattern = f"{backslash_name}"
    return regex.compile(pattern)

def _argument_detection(group_num: int):
    return "\{((?>[^{}]+|\{(?1)\})*)\}".replace("1", str(group_num))
    

In [ ]:
# Basic
pattern = regex_pattern_detecting_command(('Sur', 0, None, r'\mathrm{Sur}'))
text = r'The number of element of $\Sur(\operatorname{Cl} \mathcal{O}_L, A)$ is ...'
match = pattern.search(text)
start, end = match.span()
test_eq(text[start:end], r'\Sur')

# One parameter
pattern = regex_pattern_detecting_command(('field', 1, None, r'\mathbb{#1}'))
text = r'\field{Q}'
# print(pattern.pattern)
match = pattern.search(text)
start, end = match.span()
test_eq(text[start:end], text)

# Multiple parameters
pattern = regex_pattern_detecting_command(('mat', 4, None, r'\left[\begin{array}{cc}#1 & #2 \\ #3 & #4\end{array}\right]'))
text = r'\mat{{123}}{asdfasdf{}{}}{{{}}}{{asdf}{asdf}{}}' # This is a balanced str.
match = pattern.search(text)
start, end = match.span()
test_eq(text[start:end], text)
test_eq(match.group(1), r'{123}')

# Multiple parameters, one of which is optional parameter
pattern = regex_pattern_detecting_command(('plusbinomial', 3, '2', r'(#2 + #3)^#1'))
# When the optional parameter is used
text = r'\plusbinomial{x}{y}'
match = pattern.search(text)
start, end = match.span()
test_eq(text[start:end], text)

# When the optional parameter is not used
text = r'\plusbinomial[4]{x}{y}'
match = pattern.search(text)
start, end = match.span()
test_eq(text[start:end], text)

# One parameter that is optional.
pattern = regex_pattern_detecting_command(('greet', 1, 'world', r'Hello #1!'))
# When the optional parameter is used
text = r'\greet'
match = pattern.search(text)
start, end = match.span()
test_eq(text[start:end], text)

# When the optional parameter is not used
text = r'\greet[govna]'
match = pattern.search(text)
start, end = match.span()
test_eq(text[start:end], text)

In [ ]:
#| export
def replace_command_in_text(
        text: str,
        command_tuple: tuple[str, int, Union[None, str], str], # Consists of 1. the name of the custom command 2. the number of parameters 3. The default argument if specified or `None` otherwise, and 4. the display text of the command.
    ):
    """
    Replaces all invocations of the specified command in `text` with the display text
    with the arguments used in the display text.

    Assumes that '\1', '\2', '\3', etc. are not part of the display text. 
    """
    command_name, num_parameters, optional_arg, display_text = command_tuple
    command_pattern = regex_pattern_detecting_command(command_tuple)
    replace_pattern = display_text.replace('\\', r'\\')
    # if optional_arg is not None:
    #     replace_pattern = replace_pattern.replace('#1', optional_arg)
    replace_pattern = re.sub(r'#(\d)', r'\\\1', replace_pattern)
    text = regex.sub(
        command_pattern,
        lambda match: _replace_command(match, command_tuple, command_pattern, replace_pattern),
        text)
    return text
    # if optional_arg is not None:
    #     trailing_arguments = [_argument_detection(i) for i in range(2, 1+num_parameters)]
    #     trailing_args_pattern = "\\s*".join(trailing_arguments)
    #     pattern = (f"{pattern}\\s*{trailing_args_pattern}")
    # elif num_parameters > 0:
    #     arguments = [_argument_detection(i) for i in range(1, 1+num_parameters)]
    #     args_pattern = "\\s*".join(arguments)
    #     pattern = f"{backslash_name}\\s*{args_pattern}"
    # else:
    #     pattern = f"{backslash_name}"
    # return regex.compile(pattern)

def _replace_command(
        match: regex.match,
        command_tuple: [str, int, Union[None, str], str],
        command_pattern: regex.Pattern,
        replace_pattern: re.Pattern) -> str:
    """Replace the matched command with the display text"""
    command_name, num_parameters, optional_arg, display_text = command_tuple
    start, end = match.span()
    matched_string_to_replace = match.string[start:end]
    if len(match.groups()) > 0 and match.group(1) is None:
        replace_pattern = replace_pattern.replace(r'\1', optional_arg)
        replaced_string = regex.sub(command_pattern, replace_pattern, matched_string_to_replace)
        return replaced_string
    else:
        return regex.sub(command_pattern, replace_pattern, matched_string_to_replace)



In [ ]:
# Basic
command_tuple = ('Sur', 0, None, r'\mathrm{Sur}')
pattern = regex_pattern_detecting_command(command_tuple)
text = r'The number of element of $\Sur(\operatorname{Cl} \mathcal{O}_L, A)$ is ... Perhaps $\Sur$ is nonempty.'
test_eq(replace_command_in_text(text, command_tuple), 'The number of element of $\mathrm{Sur}(\operatorname{Cl} \mathcal{O}_L, A)$ is ... Perhaps $\mathrm{Sur}$ is nonempty.')


# One parameter
command_tuple = ('field', 1, None, r'\mathbb{#1}')
pattern = regex_pattern_detecting_command(command_tuple)
text = r'$\field{Q}$ is the field of rational numbers. $\field{C}$ is the field of complex numbers'
test_eq(replace_command_in_text(text, command_tuple), '$\mathbb{Q}$ is the field of rational numbers. $\mathbb{C}$ is the field of complex numbers')

# Multiple parameters
command_tuple = ('mat', 4, None, r'\left[\begin{array}{cc}#1 & #2 \\ #3 & #4\end{array}\right]')
pattern = regex_pattern_detecting_command(command_tuple)
text = r'\mat{{123}}{asdfasdf{}{}}{{{}}}{{asdf}{asdf}{}}' # This is a balanced str.
test_eq(replace_command_in_text(text, command_tuple), r'\left[\begin{array}{cc}{123} & asdfasdf{}{} \\ {{}} & {asdf}{asdf}{}\end{array}\right]')

# Multiple parameters, one of which is optional parameter
command_tuple = ('plusbinomial', 3, '2', r'(#2 + #3)^#1')
pattern = regex_pattern_detecting_command(command_tuple)
# When the optional parameter is used
text = r'\plusbinomial{x}{y}'
test_eq(replace_command_in_text(text, command_tuple), r'(x + y)^2')

# When the optional parameter is not used
text = r'\plusbinomial[4]{x}{y}'
test_eq(replace_command_in_text(text, command_tuple), r'(x + y)^4')


# One parameter that is optional.
command_tuple = ('greet', 1, 'world', r'Hello #1!')
pattern = regex_pattern_detecting_command(command_tuple)
# When the optional parameter is used
text = r'\greet'
test_eq(replace_command_in_text(text, command_tuple), r'Hello world!')

# When the optional parameter is not used
text = r'\greet[govna]'
test_eq(replace_command_in_text(text, command_tuple), r'Hello govna!')

In [ ]:
#| export 
def replace_commands_in_latex_document(
        docment: str
        ) -> str:
    """Return the latex document (without the preamble) with invocations
    of custom commands/operators replaced with their display text.

    Assumes that all custom commands and operators are defined in the
    preamble.

    Assumes that, if commands with the same name are defined multiple times,
    only the finally defined command is used. 

    Even replaces these invocations incommented out text.
    """
    preamble, document = divide_preamble(docment)
    commands = custom_commands(preamble)
    # Note that `command_tuple[0]` is the name of the command.
    unique_commands = {command_tuple[0]: command_tuple for command_tuple in commands} 
    for _, command_tuple in unique_commands.items():
        document = replace_command_in_text(document, command_tuple)
    return document
    

In [ ]:
file = _test_directory() / 'latex_examples' / 'commands_recursive_example' / 'main.tex'
document = text_from_file(file)
commands_replaced = replace_commands_in_latex_document(document)
assert commands_replaced.startswith(r'\begin{document}')
assert commands_replaced.endswith(r'\end{document}')
assert r'\S' not in commands_replaced
assert r'\mathbb{S}1' in commands_replaced  # Note that $\S$ is defined twice in the preamble; only the latter definition is used.
assert r'\field{Q}$' not in commands_replaced
assert r'\mathbb{Q}$' in commands_replaced
assert r'\commentedout' not in commands_replaced
assert r'This is actually a command that is commented out, but it is also replaced!' in commands_replaced
print(commands_replaced)

\begin{document}

$\mathbb{S}1$
%$\mathbf{Q}$
%$\mathbf{Q}$
%This is actually a command that is commented out, but it is also replaced!
$\mathbb{Q}$

\end{document}
